In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic #Idiosyncratic means unique to an individual
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [198]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [195]:
type(sentences[0][0])

spacy.tokens.span.Span

Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [7]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,rejoice,argue,mourning,omit,grunt,building,mad,alter,accident,different,...,effect,offended,endeavour,limb,this,utmost,improvement,fill,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [9]:
type(word_counts)
word_counts.to_csv('word_counts.csv')

## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [10]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

C:\Users\Ruecker-PC\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Training set score: 0.987460815047022

Test set score: 0.8853383458646616


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [12]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8846394984326019

Test set score: 0.8735902255639098


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [13]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
print(emma[:100])
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a
Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [14]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [15]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [16]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [17]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6976137211036539


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

## Feature generation

In [86]:
def pos_features(sentences):
    
    possiblePOS=['POS','ADJ','ADP','ADV','AUX','CONJ','CCONJ','DET','INTJ','NOUN','NUM',
             'PART','PRON','PROPN','PUNCT','SCONJ','SYM','VERB','X','SPACE']
    df_sent = pd.DataFrame(sentences)
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=possiblePOS)
    df['text_sentence'] = df_sent[0]
    df['text_source'] = df_sent[1]
    df.loc[:, possiblePOS] = 0
    
    # Process each row, counting the occurrence of POS in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        # Convert the sentence to doc and then extract all pos
        str_sent = str(sentence)
        doc = nlp(str_sent)
        for token in enumerate(doc):
            #print('token: ',token[1])
            #print('token_pos: ',token[1].pos_)
            poss = [token.pos_ for token in doc]
        
        # Populate the row with pos counts.
        for pos in poss:
            df.loc[i, pos] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
    # replace all NaN with 0  
    df=df.fillna(0) 

    # some more features
    df['NounsNonNounsRatio']=df['NOUN']/df.sum(axis=1)
    df['NounsVerbsRatio']=df['NOUN']/(df['NOUN']+df['VERB'])
    df['ADJ+ADVRatio'] = (df['ADJ']+df['ADV'])/df.sum(axis=1)
    
    # Fix some issues of the df
    df=df.fillna(0) # replace all NaN with 0
    return df

In [196]:
type(alice_sents)

list

In [197]:
type(alice_sents[0][0])

spacy.tokens.span.Span

In [88]:
alice_pos_features=pos_features(alice_sents)
persuation_pos_features= pos_features(persuasion_sents)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500


In [118]:
# combine the word and the POS_dfs
# 1. Combine the POS_df
POS_both = alice_pos_features.append(pd.DataFrame(data = persuation_pos_features), ignore_index=True)
print(POS_both.shape)

(5318, 25)


In [108]:
possiblePOS_minusSpacePUNCT=['POS','ADJ','ADP','ADV','AUX','CONJ','CCONJ','DET','INTJ','NOUN','NUM',
             'PART','PRON','PROPN','SCONJ','SYM','VERB','X']
# Some by hand Feature engineering
print('Before:', POS_both.shape)
# sentence length
POS_both['SentenceLength'] = POS_both[possiblePOS_minusSpacePUNCT].sum(axis=1)
print('After:', POS_both.shape)
POS_both.head()
# Previous Sentence Length
previousLen=[]
for i, item in enumerate(POS_both['SentenceLength']):
    if i > 0:
        previousLen.append(POS_both['SentenceLength'][i-1])
    else:
         previousLen.append(0)
POS_both['LengthPreviousSentence']=previousLen
print('After:', POS_both.shape)

Before: (5318, 26)
After: (5318, 26)
After: (5318, 27)


In [112]:
word_counts=word_counts.drop(['SentenceLength', 'LengthPreviousSentence'], axis=1)

In [113]:
#2. Add POS_df columns to the words_df
print('Before join:', word_counts.shape)

words_pos= word_counts.join(POS_both.set_index(['text_sentence', 'text_source']), on=['text_sentence', 'text_source'])

print('After:', words_pos.shape)
words_pos.head()

Before join: (5318, 3064)
After: (5318, 3089)


,rejoice,argue,mourning,omit,grunt,building,mad,alter,accident,different,...,SCONJ,SYM,VERB,X,SPACE,NounsNonNounsRatio,NounsVerbsRatio,ADJ+ADVRatio,SentenceLength,LengthPreviousSentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,13,0,0,0.179104,0.480000,0.088680,57,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,11,0,0,0.126984,0.421053,0.220306,56,57
2,0,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0.060606,0.285714,0.239907,29,56
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000000,0.000000,0.333333,2,29
4,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0.000000,0.000000,0.166667,4,2


## Model with ~25 more POS features, plus several stat features

In [114]:
Y2 = words_pos['text_source']
X2 = np.array(words_pos.drop(['text_sentence','text_source'], axis=1))

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, 
                                                    Y2,
                                                    test_size=0.4,
                                                    random_state=0)

In [115]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()

train = rfc.fit(X2_train, y2_train)

print('Training set score:', rfc.score(X2_train, y2_train))
print('\nTest set score:', rfc.score(X2_test, y2_test))

Training set score: 0.9896551724137931

Test set score: 0.855733082706767


In [116]:
lr = LogisticRegression()
train = lr.fit(X2_train, y2_train)
print('Training set score:', lr.score(X2_train, y2_train))
print('\nTest set score:', lr.score(X2_test, y2_test))

Training set score: 0.9605015673981191

Test set score: 0.9196428571428571


In [117]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X2_train, y2_train)

print('Training set score:', clf.score(X2_train, y2_train))
print('\nTest set score:', clf.score(X2_test, y2_test))

Training set score: 0.8949843260188087

Test set score: 0.8754699248120301


In [120]:
y2_predicted = lr.predict(X2_test)
pd.crosstab(y2_test, y2_predicted)

col_0,Austen,Carroll
text_source,,
Austen,1431,41
Carroll,130,526


## Result Challenge 0

After using spacy to add the "parts-of-speech" attributes and the length of the sentence and the length of the privous sentence, logistic recression accuracy reaches 0.919 compared to previously 0.915.

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.
## Preparing Chesterton - The Ball and the Cross

In [264]:
ball = gutenberg.raw('chesterton-ball.txt')

# Print the first 100 characters of The Ball and The Cross.
print('\nRaw:\n', ball[0:100])

# Remove the Chapter indicator

regex = r'([XVI].{0,4}\. [A-Z].*)'

ball = re.sub(regex,'', ball)
# works here almost at least https://regex101.com/
# Print the first 100 characters of The Ball and The Cross.
print('\nTitle removed:\n', ball[0:100])


Raw:
 [The Ball and The Cross by G.K. Chesterton 1909]


I. A DISCUSSION SOMEWHAT IN THE AIR

The flying s

Title removed:
 [The Ball and The Cross by G.K. Chesterton 1909]




The flying ship of Professor Lucifer sang throu


In [142]:
ball_noT = text_cleaner(ball_noT)
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
ball_doc = nlp(ball_noT)

# Group into sentences.
ball_sents = [[sent, "Chesterton"] for sent in ball_doc.sents]
# Set up the bags.
ballwords = bag_of_words(ball_doc)

# Combine the sentences from the two novels into one data frame.
sentences3 = pd.DataFrame(alice_sents + persuasion_sents + ball_sents)

# Combine bags to create a set of unique words.
common_words3 = set(alicewords + persuasionwords + ballwords)

In [22]:
# Create our data frame with features. This can take a while to run.
word_counts3 = bow_features(sentences3, common_words3)
word_counts3.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500
Processing row 7000
Processing row 7500
Processing row 8000
Processing row 8500
Processing row 9000
Processing row 9500


,rejoice,argue,mourning,coincidence,screw,genially,german,contract,omit,cavern,...,offended,endeavour,limb,this,utmost,improvement,fill,split,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [35]:
type(word_counts3)
word_counts3.to_csv('word_counts3.csv')

In [218]:
word_counts3.shape

(9974, 3837)

In [231]:
# Loading word counts instead od recalculating
word_counts3 = pd.read_csv('word_counts3.csv')

In [144]:
# Adding the POS features
ball_pos_features=pos_features(ball_sents)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


In [243]:
# Combining Carrol, Austin and Chesterton POS
# 1. Combine the POS_df
POS_CAC = POS_both.append(pd.DataFrame(data = ball_pos_features), ignore_index=True)

# adding some additional features by hand
possiblePOS_minusSpacePUNCT=['POS','ADJ','ADP','ADV','AUX','CONJ','CCONJ','DET','INTJ','NOUN','NUM',
             'PART','PRON','PROPN','SCONJ','SYM','VERB','X']

# Some by hand Feature engineering
print('Before:', POS_CAC.shape)
# sentence length
POS_CAC['SentenceLength'] = POS_CAC[possiblePOS_minusSpacePUNCT].sum(axis=1)
print('After 1 addition:', POS_CAC.shape)
POS_CAC.head()
# Previous Sentence Length
previousLen=[]
for i, item in enumerate(POS_CAC['SentenceLength']):
    if i > 0:
        previousLen.append(POS_CAC['SentenceLength'][i-1])
    else:
         previousLen.append(0)
POS_CAC['LengthPreviousSentence']=previousLen
print('Final:', POS_CAC.shape)

Before: (9946, 25)
After 1 addition: (9946, 26)
Final: (9946, 27)


In [246]:
# Combining the POS data and the words data
word_counts3['text_sentence']=word_counts3['text_sentence'].astype(str)
word_counts3['text_source']=word_counts3['text_source'].astype(str)
POS_CAC['text_sentence']=POS_CAC['text_sentence'].astype(str)
POS_CAC['text_source']=POS_CAC['text_source'].astype(str)

WP_df = pd.merge(POS_CAC, word_counts3,  how='left', left_on=['text_sentence', 'text_source'], right_on = ['text_sentence', 'text_source'])
print(WP_df.shape)

WP_df_nona=WP_df.dropna(axis=0)
WP_df_nona=WP_df_nona.drop_duplicates()
print(WP_df_nona.shape)

(11374, 3863)
(10813, 3863)


In [255]:
WP_df_nona['text_source'].unique()

array(['Carroll', 'Austen', 'Chesterton'], dtype=object)

## Preparing data for modeling

In [267]:
# Alice and Ball
AliceBall= WP_df_nona[WP_df_nona['text_source']!="Austen"]
print(AliceBall.shape)
PersBall= WP_df_nona[WP_df_nona['text_source']!="Carroll"]
print(PersBall.shape)
AlicePers= WP_df_nona[WP_df_nona['text_source']!="Chesterton"]
print(AlicePers.shape)


#Preparing data for modeling
Y_AP = AlicePers['text_source']
X_AP = np.array(AlicePers.drop(['text_sentence','text_source'], 1))

Xap_train, Xap_test, yap_train, yap_test = train_test_split(X_AP, 
                                                    Y_AP,
                                                    test_size=0.4,
                                                    random_state=0)




#Preparing data for modeling
Y_AB = AliceBall['text_source']
X_AB = np.array(AliceBall.drop(['text_sentence','text_source'], 1))

Xab_train, Xab_test, yab_train, yab_test = train_test_split(X_AB, 
                                                    Y_AB,
                                                    test_size=0.4,
                                                    random_state=0)


#Preparing data for modeling
Y_PB = PersBall['text_source']
X_PB = np.array(PersBall.drop(['text_sentence','text_source'], 1))

Xpb_train, Xpb_test, ypb_train, ypb_test = train_test_split(X_PB, 
                                                    Y_PB,
                                                    test_size=0.4,
                                                    random_state=0)

(6954, 3863)
(9092, 3863)
(5580, 3863)


## Alice versus Persuasion

In [269]:
lr = LogisticRegression()
train = lr.fit(Xap_train, yap_train)
print('Training set score:', lr.score(Xap_train, yap_train))
print('\nTest set score:', lr.score(Xap_test, yap_test))

Training set score: 0.997610513739546

Test set score: 0.9937275985663082


## Alice versus Ball

In [270]:
lr = LogisticRegression()
train = lr.fit(Xab_train, yab_train)
print('Training set score:', lr.score(Xab_train, yab_train))
print('\nTest set score:', lr.score(Xab_test, yab_test))

Training set score: 0.9801054650047939

Test set score: 0.9816678648454349


## Persuation versus Ball

In [271]:
lr = LogisticRegression()
train = lr.fit(Xpb_train, ypb_train)
print('Training set score:', lr.score(Xpb_train, ypb_train))
print('\nTest set score:', lr.score(Xpb_test, ypb_test))

Training set score: 0.8951420714940421

Test set score: 0.8941435248831454


## Result Challenge 1

The model is particularly good in filtering out 'Alice in Wonderland'. The accuracy for differentiating both other books from 'Alice' are above 0.98. Where as, 'persuasion' and 'ball' can only be differentiated with an accuracy of 
Interestingly the model is much better in differentiating Carroll's 'Alice in Wonderland' from Chestertons 'The Ball and The Cross'. I have neither read 'Persuation', nor 'The Ball and the Cross', 